# Домашнее задание к лекции "Основы веб-скрапинга"
## Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [2]:
import pandas as pd
from numpy import NaN
import requests
from bs4 import BeautifulSoup
import time

In [3]:
def web_scraping():
    
    urls = ['https://habr.com/ru/search/page' + str(x) for x in range(1,6)]
    
    keywords = ['python', 'анализ данных']
    df_list = []
    
    for keyword in keywords:
        
      req1 = requests.get(urls[1], params={'q':[keyword]})
      req2 = requests.get(urls[2], params={'q':[keyword]})
      req3 = requests.get(urls[3], params={'q':[keyword]})
      req4 = requests.get(urls[4], params={'q':[keyword]})
      
      soup1 = BeautifulSoup(req1.text)
      soup2 = BeautifulSoup(req2.text)
      soup3 = BeautifulSoup(req3.text)
      soup4 = BeautifulSoup(req4.text)
      
      articles1 = soup1.find_all('article', class_='tm-articles-list__item')
      articles2 = soup2.find_all('article', class_='tm-articles-list__item')
      articles3 = soup3.find_all('article', class_='tm-articles-list__item')
      articles4 = soup4.find_all('article', class_='tm-articles-list__item')
      
      articles_all = articles1 + articles2 + articles3 + articles4
      time.sleep(0.2)
    
      for article in articles_all:
        title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
        if title:
          title = title.text
          link = article.find('a', class_='tm-article-snippet__title-link').get('href')
          dates = article.find('span', class_='tm-article-datetime-published').text
          likes = article.find('div', class_ = "tm-votes-meter tm-data-icons__item").text
          article_soup = BeautifulSoup(requests.get('https://habr.com' + link).text)
          texts = article_soup.find_all('div', class_='tm-article-body')[0].text
          #print(texts)
        else:
          title = NaN
          link = NaN
          dates = NaN
          article_soup = NaN
          texts = NaN
        
        df = pd.DataFrame()
        df_list.append(pd.DataFrame({'title': [title],'date': [dates], 'link': [link], 'like': [likes], 'texts': [texts]}))
      df = pd.concat(df_list).drop_duplicates()
        
    return df.reset_index(drop=True).dropna(subset=['title'])

web_scraping().head(10)

,title,date,link,like,texts
0,Юбилейный поток Python для инженеров,14 дек 2022 в 18:01,/ru/company/southbridge/news/t/705358/,Всего голосов 9: ↑7 и ↓2 +5,16 января весь Слёрм будет праздновать: старт...
1,Астрономам порекомендовали меньше использовать...,16 сен 2020 в 16:48,/ru/news/t/519414/,Всего голосов 26: ↑24 и ↓2 +22,\n\r\nАстрономы из Лейденской обсерватории оп...
2,Разработчик предложил устроить коммунистическу...,15 янв 2022 в 19:00,/ru/news/t/645777/,Всего голосов 27: ↑17 и ↓10 +7,\n\r\n14 января 2022 года разработчик jokteur...
3,Слёрм запускает 3-дневный интенсив по Python д...,18 мая 2022 в 18:33,/ru/company/southbridge/news/t/666476/,Всего голосов 7: ↑6 и ↓1 +5,24-26 июня пройдёт онлайн-интенсив для инжене...
4,3-месячный курс по Python,12 янв в 10:21,/ru/company/southbridge/news/t/710188/,Всего голосов 9: ↑8 и ↓1 +7,Рутинная работа уходит в прошлое: сегодня все...
5,24 марта Слёрм проведёт открытый урок «Первый ...,18 мар 2022 в 18:31,/ru/company/southbridge/news/t/656419/,Всего голосов 20: ↑17 и ↓3 +14,"Думаете, написать свою первую программу на Py..."
6,TechnoMeetsPython. Онлайн митап о Python-разра...,22 апр 2022 в 14:42,/ru/news/t/662437/,Всего голосов 2: ↑2 и ↓0 +2,27 апреля в 18:00 собираем питонистов на YouT...
7,Последний поток Python для инженеров в этом году,3 авг 2022 в 15:23,/ru/company/southbridge/news/t/680662/,Всего голосов 8: ↑8 и ↓0 +8,29 августа мы запускаем 4 поток курса «Python...
8,Асинхронное программирование на Python для джу...,29 сен 2022 в 09:34,/ru/company/southbridge/news/t/690692/,Всего голосов 15: ↑8 и ↓7 +1,Как джуну выделиться на фоне таких же новичко...
9,«Разработчикам вход запрещен» или курс «Python...,27 сен 2021 в 09:03,/ru/company/southbridge/news/t/580100/,Всего голосов 13: ↑12 и ↓1 +11,Полгода назад мы анонсировали первый поток ку...


![](2.png)